Bài viết tham khảo tại [nguồn](https://www.pyimagesearch.com/2019/10/28/3-ways-to-create-a-keras-model-with-tensorflow-2-0-sequential-functional-and-model-subclassing)

#### 1. Thêm các thư viện cần thiết

In [81]:

from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import concatenate

#### 2. Thiết kế model

In [109]:
class MiniVGGModel(Model):
    def __init__(self, classes):

        super(MiniVGGModel, self).__init__()
    
        # Block 1
        self.conv1A = Conv2D(64, (3, 3), padding="same")
        self.act1A = Activation("relu")
        self.bn1A = BatchNormalization()
        
        self.conv1B = Conv2D(64, (3, 3), padding="same")
        self.act1B = Activation("relu")
        self.bn1B = BatchNormalization()
        
        self.pool1 = MaxPooling2D(pool_size=(2,2))
        
        
        # Block 2. Padding Same nghĩa là duy trì kích thước đầu ra tương tự đầu vào, phần bị lược bỏ bằng Kernel sẽ 
        # được thay thế bằng 0.
        
        self.conv2A = Conv2D(32, (3, 3), padding="same")
        self.act2A = Activation("relu")
        self.bn2A = BatchNormalization()
        
        self.conv2B = Conv2D(32, (3, 3), padding="same")
        self.act2B = Activation("relu")
        self.bn2B = BatchNormalization()
        
        self.pool2 = MaxPooling2D(pool_size=(2,2))
        
        # 2 Block này gần giống nhau dùng Conv2D để trích xuất đặc trưng của anh và loại bổ data không cần thiết.
        # Đầu ra của hàm Activation (Phi tuyến) sẽ được chuẩn hoá lại BatchNorm nhằm đảm bảo qua mỗi block, phân
        # bố của anh không bị thay đổi quá nhiều
        
        
        self.flatten = Flatten()
        self.dense3 = Dense(512)
        self.act3 = Activation("relu")
        self.bn3 = BatchNormalization()
        self.do3 = Dropout(0.5)
        
        # Sau khi qua mạng trích xuất đặc trưng data được duỗi thành cỡ 512*1 và đưa vào một neural network không 
        # kết nối đầy đủ với tỉ lệ Dropout là 0.5 (Kỹ thuật Dropout là kỹ thuật tắt một số lượng nút trong mạng bất kỳ)
        # dựa trên một tỉ lệ cho trước
        
        self.dense4 = Dense(classes)
        self.softmax = Activation("softmax")
        
        # Mạng Neural Network này có đầu ra là một lớp Softmax với 10 nhãn cho trước.
        
    
    def call(self, inputs):
        
        # Block 1. Lần lượt đưa dữ liệu qua các lớp.
        
        x = self.conv1A(inputs)
        x = self.act1A(x)
        x = self.bn1A(x)
    
        x = self.conv1B(x)
        x = self.act1B(x)
        x = self.bn1B(x)
        x = self.pool1(x)
        
        
        # Block 2
        
        x = self.conv2A(x)
        x = self.act2A(x)
        x = self.bn2A(x)
        
        x = self.conv2B(x)
        x = self.act2B(x)
        x = self.bn2B(x)
        
        x = self.pool2(x)
        
        # Block 3
        
        x = self.flatten(x)
        x = self.dense3(x)
        x = self.act3(x)
        x = self.bn3(x)
        x = self.do3(x)
        
        
        # Block 4
        
        x = self.dense4(x)
        x = self.softmax(x)
        
        return x

#### 3.Tải dữ liệu và chuẩn hoá

<img src="./img/cifa.png" width=600/>

In [96]:
from tensorflow.keras.datasets import cifar10

In [97]:
((trainX, trainY), (testX, testY)) = cifar10.load_data()

Đưa dữ liệu về trong khoảng (0,1). Tại sao phải làm như thế này thì tôi đã giải thích tại bài trước. [Xem ở đây](./setup-and-first-code.ipynb)

In [98]:
trainX = trainX.astype("float32") / 255.0
testX = testX.astype("float32") / 255.0

Nhãn được đánh từ 0 đến 9 tương đương với danh sách dưới đây

In [99]:
labelNames = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

Chuyển về dạng one hot.

One hot chỉ là cách chuyển nhãn về sao cho đảm bảo tính tương đồng của các nhãn với nhau. Hình ảnh dưới đây minh hoạ ma trận onehot.

<img src="./img/onehot.jpeg" width=600/>

In [100]:
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD

In [101]:
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

In [102]:
testY

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 1, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0]])

#### 4. Tăng cường data khi train

Ta có thể xoay, zoom to, thay đổi kích cỡ để có nhiều đặc trưng hơn mà vẫn giữ nguyên nhãn

Xem kỹ hơn về tăng cường data [tại đây:](https://github.com/bangoc123/learn-machine-learning-in-two-months/blob/master/data-processing/ImageAugmentation.ipynb?fbclid=IwAR2WIgr3FqjGRhWaFENLiFEl4MPO4HKGfGUmRIIsM54EF_bF8_3QhsDN7rg)

In [118]:
aug = ImageDataGenerator(rotation_range=18, zoom_range=0.15, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.15, horizontal_flip=True, fill_mode="nearest")

In [104]:
model = MiniVGGModel(len(labelNames))

#### 5. Đặt các hyperparameter

In [105]:
learning_rate = 1e-2
batch_size = 64
num_epochs = 30 # Accuracy: 74% within 30 epochs

Sử dụng thuật toán tối ưu Stochastic gradient descent thay vì feed toàn bộ data trong mỗi epoch mà chia thành từng batch nhỏ.

In [106]:
opt = SGD(lr=learning_rate, momentum=0.9, decay=learning_rate / num_epochs)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

Tiến hành training

In [ ]:
print("Start Training...")
H = model.fit_generator(aug.flow(trainX, trainY, batch_size=batch_size), validation_data=(testX, testY), 
                        steps_per_epoch=trainX.shape[0] // batch_size, epochs=num_epochs, verbose=1)

Save model lại.

In [115]:
model.save("final", save_format='tf')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: final/assets


#### 6 Kết quả và Inference

Tôi đang train trên Colab gần xong rồi, khi nào có kết quả sẽ viết tiếp phần Inference. Độ chính xác đang là 77%

<img src="./img/training.png" width=600/>